In [8]:
%load_ext Cython


In [10]:
import numpy as np

def pt2LineDis(Pt, Lpt1, Lpt2):
    q1=Lpt1-Pt
    q2=Lpt2-Pt
    q3=Lpt1-Lpt2
    return np.cross(q1,q2)/np.linalg.norm(q3)

def pts2LineDisWithRef(Pts, PtPosRef, Lpt1, Lpt2):
    DisRes = np.array([pt2LineDis(Pt, Lpt1, Lpt2) for Pt in Pts])
    if pt2LineDis(PtPosRef, Lpt1, Lpt2) < 0:
        DisRes = -DisRes
    return DisRes

p1 = np.array([1,0])
p2 = np.array([-1,0])

p3 = np.array([1,1.5])

pts2LineDisWithRef(
    Pts=[np.array([1,2]), np.array([-1,2.5]), np.array([-1,-2.5])],
    PtPosRef = np.array([1,99]),
    Lpt1=p1,
    Lpt2=p2
)


# pt2LineDis(Pt=p3, Lpt1=p1, Lpt2=p2)


array([ 2. ,  2.5, -2.5])

In [11]:
import numpy as np
import matplotlib.pyplot as plt

# this sets up the Matplotlib interactive windows:
%matplotlib widget
# this changes the default date converter for better interactive plotting of dates:
plt.rcParams['date.converter'] = 'concise'

#   Note the use of datetimes in the file complicate loading a bit.
#   We recommend using pandas or xarray for more elegant solutions
#   to handling complex timeseries data. 
with open('RECYCLE/small.txt', 'r') as f:
    data = np.genfromtxt(f, dtype='datetime64[s],f,f,f', 
                         names=['date', 'doy', 'temp', 'solar'])
datetime = data['date']
dayofyear = data['doy']
temperature = data['temp']
solar = data['solar']

# make two-day smoothed versions:
temp_low = np.convolve(temperature, np.ones(48)/48, mode='same')
solar_low = np.convolve(solar, np.ones(48)/48, mode='same')

fig, (ax0, ax1) = plt.subplots(2, 1, sharex=True, constrained_layout=True)

# temperature:
ax0.plot(datetime, temperature, label='hourly')
ax0.plot(datetime, temp_low, label='smoothed')
ax0.legend(loc='upper right')
ax0.set_ylabel('Temperature $[^oC]$')  # note the use of TeX math formatting

# solar-radiation:
ax1.plot(datetime, solar, label='hourly')
ax1.plot(datetime, solar_low, label='smoothed')
ax1.legend(loc='upper right')
ax1.set_ylabel('Solar radiation $[W\,m^{-2}]$')   # note the use of TeX math formatting

ax0.set_title('Observations: Dinosaur, Colorado', loc='left')
ax0.text(0.03, 0.03, 'https://www.ncei.noaa.gov/pub/data/uscrn/products/hourly02/', 
         fontsize='small', fontstyle='italic', transform=ax0.transAxes);

FileNotFoundError: [Errno 2] No such file or directory: 'data/small.txt'

In [9]:
%%cython

a: cython.int = 0
for i in range(10):
    a += i
print(a)

45


In [73]:
%%cython
import numpy as np
cimport numpy as np
cimport cython
from cython.parallel import prange
from libc.math cimport cos, atan2, abs

np.import_array()
ctypedef np.float64_t DTYPE_F64_t
ctypedef np.float32_t DTYPE_t
ctypedef np.int64_t DTYPE_int64_t
ctypedef np.uint8_t DTYPE_bool_t

PI = np.pi
@cython.boundscheck(False)
@cython.wraparound(False)
cdef reg_rad(DTYPE_F64_t rad):
    return (rad + PI) % (2 * PI) - PI

@cython.boundscheck(False)
@cython.wraparound(False)
cdef laser_hit_improve3(np.ndarray[DTYPE_F64_t, ndim=1] pos_o, np.ndarray[DTYPE_F64_t, ndim=1] pos_t, 
    DTYPE_F64_t fanRadius, DTYPE_F64_t fanOpenRad, DTYPE_F64_t fanDirRad):

    cdef DTYPE_F64_t dis_square
    cdef np.ndarray[DTYPE_F64_t, ndim=1] delta

    cdef DTYPE_F64_t ori_rad_pos, ori_rad_neg, ori_2tgt

    delta = pos_t - pos_o
    dis_square = delta[0]*delta[0] + delta[1]*delta[1]

    if dis_square > fanRadius*fanRadius: return False
    
    ori_rad_pos = fanDirRad + fanOpenRad/2
    ori_rad_neg = fanDirRad - fanOpenRad/2
    
    ori_2tgt = atan2(delta[1],delta[0])
    
    d1rad = abs(reg_rad(ori_rad_pos - ori_2tgt))
    d2rad = abs(reg_rad(ori_rad_neg - ori_2tgt))
    
    if d1rad <= fanOpenRad and d2rad <= fanOpenRad:
        return True
    else:
        return False

    

In [75]:
pos_o = np.array([0.0, 0.0])
fanRadius = 2.0
fanOpenRad = PI/2 # 90 deg
fanDirRad =  0 # ? deg

for theta in np.arange(-180, 180, 10):
    theta_rad = theta*PI/180
    pos_t = np.array([np.cos(theta_rad), np.sin(theta_rad)])
    print(theta, laser_hit_improve3(pos_o, pos_t, fanRadius, fanOpenRad, fanDirRad))

-180 False
-170 False
-160 False
-150 False
-140 False
-130 False
-120 False
-110 False
-100 False
-90 False
-80 False
-70 False
-60 False
-50 False
-40 True
-30 True
-20 True
-10 True
0 True
10 True
20 True
30 True
40 True
50 False
60 False
70 False
80 False
90 False
100 False
110 False
120 False
130 False
140 False
150 False
160 False
170 False


atan2(y,x)

In [42]:
import numpy as np
deg = [ np.arccos(i/100)*np.pi/180  for i in range(-100,100)]
deg

[0.05483113556160754,
 0.05236080573055761,
 0.051334632956605425,
 0.05054520875160189,
 0.04987799395815786,
 0.04928866049642385,
 0.04875449214682576,
 0.048262004545041336,
 0.04780241817608512,
 0.047369638945661235,
 0.04695923268098997,
 0.046567854019832496,
 0.04619290547647732,
 0.04583232254906506,
 0.04548443221163199,
 0.04514785630971444,
 0.04482144358609045,
 0.044504220604608936,
 0.04419535552319032,
 0.04389413082991245,
 0.0435999224729305,
 0.04331218364235137,
 0.04303043199658701,
 0.04275423947943935,
 0.04248322411347368,
 0.042217043320338814,
 0.04195538843463453,
 0.04169798016066698,
 0.041444564781350696,
 0.04119491097249394,
 0.04094880710838298,
 0.04070605896914274,
 0.04046648777900686,
 0.040229928518940125,
 0.0399962284681325,
 0.03976524593753071,
 0.039536849165377336,
 0.03931091535012165,
 0.03908732980037239,
 0.03886598518502331,
 0.03864678086948082,
 0.0384296223261989,
 0.03821442060958808,
 0.038001091886896154,
 0.037789557017922824,
 0

In [ ]:
confidence_threshhold = 0.25*1.05
eye_mat = np_repeat_at(np.eye(n_agent), insert_dim=0, n_times=n_timestep)
compasion_matrix = correct_prediction & (prediction_rating > confidence_threshhold) & not_zero_mask | (eye_mat==1)
print(compasion_matrix)

In [ ]:
# compasion_matrix = (prediction_rating > confidence_threshhold) & not_zero_mask & (eye_mat==0)
compasion_matrix = compasion_matrix.astype(np.int)

# 计算分解后的reward，因为reward可能被多个智能体分解
n_com = compasion_matrix.sum(-2)
assert not (n_com==0).any()
reward_decmp = np.where(n_com!=0, reward/n_com, reward) # reward/(n_com+1e-7)
# https://numpy.org/doc/stable/reference/generated/numpy.matmul.html
compassion_reward = np.matmul(compasion_matrix, np.expand_dims(reward_decmp,-1)).squeeze(-1)
print(compassion_reward)

In [ ]:
'''
函数说明：在有限的、不均衡的多标签数据集中，按照预设的比例，取出尽可能多的样本
'''
def sample_balance(x, y, n_class, weight=None):
    if weight is None: weight = torch.ones(n_class, device=x.device)
    else: weight = torch.Tensor(weight, device=x.device)
    n_instance = torch.zeros(n_class, device=x.device)
    indices = [None]*n_class
    for i in range(n_class):
        indices[i] = torch.where(y==i)[0]
        n_instance[i] = len(indices[i])
    ratio = n_instance/weight
    bottle_neck = torch.argmin(n_instance/weight)
    r = ratio[bottle_neck]
    n_sample = (r*weight).long()
    # print(n_instance, n_sample)
    new_indices = [indices[i][torch.randperm(n_sample[i])] for i in range(n_class)]
    # print(new_indices)
    new_indices_ = torch.cat(new_indices)
    assert len(new_indices_) == sum(n_sample)
    return x[new_indices_], y[new_indices_]

'''
测试代码
'''
x = torch.rand(200, 4)
y1 = torch.rand(100, 4)
y2 = torch.rand(100, 4)
y2[:, 0] += 1
y = torch.cat((y1,y2))
y = torch.argmax(y, -1)
print(y)
n_class = 4
weight = [2,1,1,1]
sample_balance(x,y,4,weight)

In [ ]:
x = torch.rand(100, 4)
y = torch.rand(100, 4)
y = torch.argmax(y, -1)

# return x[new_indices_], y[new_indices_]

In [ ]:
import numpy as np
def my_view(x, shape):
    if -1 in shape[1:-1]: return my_view_test(x, shape)
    reverse_lookup = True if shape[0] == -1 else False
    if not reverse_lookup:
        for i, dim in enumerate(shape):
            if dim == 0:
                shape[i] = x.shape[i]
    else:
        for i in range(len(shape)):
            ni = -(i + 1)  # iter -1,-2,-3,...
            dim = shape[ni]
            if dim == 0:
                shape[ni] = x.shape[ni]
    if isinstance(x, np.ndarray):
        return x.reshape(*shape)
    return x.view(*shape)

def my_view_test(x, shape):
    # fill both way until meet -1 
    for i, dim in enumerate(shape):
        if dim == 0: shape[i] = x.shape[i]
        elif dim == -1: break
    for i in range(len(shape)):
        ni = -(i + 1); dim = shape[ni]
        if dim == 0: shape[ni] = x.shape[ni]
        elif dim == -1: break
    # print(shape)
    if isinstance(x, np.ndarray):
        return x.reshape(*shape)
    return x.view(*shape)



In [ ]:

"""
    improve np.reshape and torch.view function
    If a dim is assigned with 0, it will keep its original dimension
    eg.1    x.shape = (4, 5, 6, 7); new_shape = [0, 0, -1]
            y = my_view(x, new_shape)
            y.shape = (4, 5, 6*7)

    eg.2    x.shape = (4, 5, 6, 7); new_shape = [-1, 0, 0]
            y = my_view(x, new_shape)
            y.shape = (4*5, 6, 7)

(4, 5, 6); new_shape = [0, 0, -1, 3]

(4, 5, 6); new_shape = [2, -1, 0, 0]
(3, 4, 5, 6); new_shape = [0, 2, -1, 0, 0]
"""

my_view(np.zeros(shape=(4, 5, 6)), shape = [2, -1, 0, 0]).shape

In [ ]:
my_view(np.zeros(shape=(4, 5, 6)), shape = [0, 0, -1, 3]).shape

In [ ]:
load_ext Cython

In [ ]:
%%cython



In [ ]:
new_method(np.ones(5), np.zeros(5))